In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import pandas as pd
from fastcore.xtras import load_pickle

from gptchem.evaluator import evaluate_photoswitch_smiles_pred

In [4]:
res = load_pickle("out/20230116_131137/summary.pkl")

In [ ]:
evaluate_photoswitch_smiles_pred(res["res_at_temp"][0]["valid_smiles"])

In [9]:
res.keys()

dict_keys(['train_size', 'noise_level', 'num_samples', 'temperatures', 'res_at_temp', 'test_size'])